In [13]:
import awkward as ak
import numpy as np


def photon_preselections_ggH_BBGG(
    photons: ak.Array,
    events: ak.Array,
    electron_veto=True,
    revert_electron_veto=False,
    year="2018",
    IsFlag=False):
    """
    Apply full preselection on leptons, jets, and photons.
    Finally return only photons from events that pass all criteria.
    """

    print("Number of events before preselection:", len(events))
    print("Year:", year)

    #-------------------------
    # b-tagging working point
    #-------------------------
    if year == "2022":
        wp_medium = 0.2783
    elif year == "2022EE":
        wp_medium = 0.2783
    else:
        raise ValueError(f"Unknown year {year}")

    # ------------------------
    # Photon selection
    # ------------------------
    abs_eta = np.abs(photons.eta)

    # Barrel–endcap transition exclusion (1.442 ≤ |η| ≤ 1.566)
    valid_eta = (abs_eta <= 2.5) & ~((abs_eta >= 1.442) & (abs_eta <= 1.566))

    # Barrel vs Endcap ID cuts
    is_barrel = abs_eta < 1.442
    is_endcap = (abs_eta > 1.566) & (abs_eta < 2.5)

    # Apply region-specific MVA thresholds
    barrel_cut = is_barrel & (photons.mvaID > -0.02)
    endcap_cut = is_endcap & (photons.mvaID > -0.26)

    # Base photon selection
    good_photons = (
        (photons.pt > 10)
        & valid_eta
        & (barrel_cut | endcap_cut)
        & (~photons.pixelSeed)
    )
    selected_photons = photons[good_photons]

    # Sort photons by pT in descending order
    selected_photons = selected_photons[ak.argsort(selected_photons.pt, ascending=False)]

    # ------------------------
    # Lead / Sublead cuts
    # ------------------------
    # Define masks for events that have at least 2 photons
    has_two_photons = ak.num(selected_photons) >= 2

    # Safe indexing: fill with -inf if photon is missing
    lead_pt = ak.fill_none(ak.firsts(selected_photons.pt), -999)
    sublead_pt = ak.fill_none(ak.pad_none(selected_photons.pt, 2)[:, 1], -999)

    lead_cut = lead_pt > 30
    sublead_cut = sublead_pt > 18

    # Event must have 2 photons and satisfy both pT cuts
    photon_event_mask = has_two_photons & lead_cut & sublead_cut
    # photon_event_mask = has_two_photons & lead_cut

    # ------------------------
    # Combine jet + photon criteria
    # ------------------------
    # event_mask = at_least_two_bjets & photon_event_mask
    event_mask = photon_event_mask

    # Prepare empty arrays for masked-out events
    empty_photons = ak.Array([[]] * len(events))

    # Keep only events passing full selection
    filtered_photons = ak.where(event_mask, selected_photons, empty_photons)
    # filtered_jets = ak.where(event_mask, selected_bjets, empty_bjets)
    # gen_bquark = ak.where(event_mask, bquarks_from_a, empty_bquarks)

    # gen_bquark = gen_bquark[ak.num(gen_bquark.pt)>0]

    print(f"Events passing full preselection: {ak.sum(event_mask)}")

    return filtered_photons

In [23]:
import awkward as ak
import numpy as np

def photon_preselections_ggH_BBGG_ROOT_like(photons, events, year="2022"):
    """
    Make Awkward photon selection identical to ROOT RDataFrame pipeline.
    """

    print("Number of events before preselection:", len(events))
    print("Year:", year)

    # ------------------------
    # 1. Sort photons by pT (descending)
    # ------------------------
    sorted_idx = ak.argsort(photons.pt, ascending=False)
    pho = photons[sorted_idx]

    # ------------------------
    # 2. Pad photon list to guarantee index 0 and 1 exist
    #    Equivalent to ROOT accessing iL and iS after nPhoton>=2 filter
    # ------------------------
    pho_padded = ak.pad_none(pho, 2)   # pad to length ≥ 2
    iL = pho_padded[:, 0]
    iS = pho_padded[:, 1]

    # ------------------------
    # 3. Require ≥2 photons BEFORE any further cuts
    # ------------------------
    has_two_photons = ak.num(pho) >= 2

    # ------------------------
    # 4. pT cuts (same as ROOT)
    # ------------------------
    pt_cut = (iL.pt > 30) & (iS.pt > 18)

    # ------------------------
    # 5. eta acceptance (ROOT exact)
    # |η| < 1.442  OR  1.556 < |η| < 2.5
    # ------------------------
    abs_eta_L = np.abs(iL.eta)
    abs_eta_S = np.abs(iS.eta)

    eta_cut_L = ((abs_eta_L < 1.442) |
                 ((abs_eta_L > 1.556) & (abs_eta_L < 2.5)))

    eta_cut_S = ((abs_eta_S < 1.442) |
                 ((abs_eta_S > 1.556) & (abs_eta_S < 2.5)))

    # ------------------------
    # 6. MVAID (ROOT ternary logic)
    # ------------------------
    mva_cut_L = ak.where(abs_eta_L < 1.442,
                         iL.mvaID > -0.02,
                         iL.mvaID > -0.26)

    mva_cut_S = ak.where(abs_eta_S < 1.442,
                         iS.mvaID > -0.02,
                         iS.mvaID > -0.26)

    # ------------------------
    # 7. Pixel seed veto
    # ------------------------
    pixel_cut = (iL.pixelSeed == 0) & (iS.pixelSeed == 0)

    # ------------------------
    # 8. Final event mask
    # ------------------------
    event_mask = (
        has_two_photons &
        pt_cut &
        eta_cut_L & eta_cut_S &
        mva_cut_L & mva_cut_S &
        pixel_cut
    )

    print("Events passing full ROOT-like preselection:", ak.sum(event_mask))

    # ------------------------
    # 9. Return photons from selected events
    # ------------------------
    empty = ak.Array([[]] * len(events))
    selected_photons = ak.where(event_mask, pho, empty)

    return selected_photons


In [6]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak

file = '/eos/user/b/bbapi/MC_contacts/ggH_Run3_private_sample/CMSSW_15_0_15_patch4/src/HAHM_15_Nano15_new.root'
factory = NanoEventsFactory.from_root(
    f"{file}:Events",
    schemaclass=NanoAODSchema,
)
events = factory.events()

events = ak.materialize(events)


In [4]:
events.fields

['GenJetAK8',
 'Muon',
 'Dataset',
 'Rho',
 'BeamSpot',
 'Electron',
 'TrkMET',
 'SubGenJetAK8',
 'PFCand',
 'DeepMETResolutionTune',
 'LHEPdfWeight',
 'PVBS',
 'FatJet',
 'PSWeight',
 'DeepMETResponseTune',
 'TrackGenJetAK4',
 'Generator',
 'LHEWeight',
 'bunchCrossing',
 'RawPuppiMET',
 'Pileup',
 'GenDressedLepton',
 'HLT',
 'MC',
 'genWeight',
 'FiducialMET',
 'TauSpinner',
 'LowPtElectron',
 'TrigObj',
 'LHEPart',
 'PFMET',
 'GenJet',
 'SoftActivityJetNjets5',
 'RawPFMET',
 'GenIsolatedPhoton',
 'SoftActivityJet',
 'GenVtx',
 'FatJetPFCand',
 'SoftActivityJetHT',
 'run',
 'SubJet',
 'SoftActivityJetNjets10',
 'TauProd',
 'orbitNumber',
 'CaloMET',
 'GenVisTau',
 'HLTriggerFinalPath',
 'OtherPV',
 'Flag',
 'HTXS',
 'FsrPhoton',
 'Tau',
 'LHEScaleWeight',
 'Jet',
 'SoftActivityJetHT10',
 'PuppiMET',
 'L1simulation',
 'L1',
 'GenProton',
 'SoftActivityJetNjets2',
 'boostedTau',
 'luminosityBlock',
 'SoftActivityJetHT5',
 'GenPart',
 'CorrT1METJet',
 'HLTriggerFirstPath',
 'LHEReweigh

In [14]:
fp = photon_preselections_ggH_BBGG(events.Photon, events,year="2022")

Number of events before preselection: 260898
Year: 2022
Events passing full preselection: 21


In [8]:
fp[ak.num(fp.pt)>0].pt

<Array [[45.3, 20.8], [...], ..., [74.8, 27.8]] type='21 * var * float32[pa...'>

In [24]:
fp_ROOT_like = photon_preselections_ggH_BBGG_ROOT_like(events.Photon, events)

Number of events before preselection: 260898
Year: 2022


Events passing full ROOT-like preselection: 8


In [26]:
fp_ROOT_like[ak.num(fp_ROOT_like.pt)>0].pt

<Array [[65.8, 22.1], [...], ..., [63.5, 47.9]] type='8 * var * ?float32[pa...'>

In [15]:
import ROOT


# Load dataset

df0 = ROOT.RDataFrame("Events", "TTtoLNu2Q_TuneCP5_13p6TeV_powheg-pythia8_2024.root")

#CUTFLOW COUNTERS
n0 = df0.Count()   # total events


# 1. Require >=2 photons

df1 = df0.Filter("nPhoton >= 2")
n1 = df1.Count()


# 2. Sort by pT

df2 = df1.Define("idx", "Reverse(Argsort(Photon_pt))") \
         .Define("iL", "idx[0]") \
         .Define("iS", "idx[1]")


# 3. pT cuts

df3 = df2.Filter("Photon_pt[iL] > 30 && Photon_pt[iS] > 18")
n2 = df3.Count()


# 4. eta acceptance (barrel + endcap)

# df4 = df3.Filter("(abs(Photon_eta[iL]) < 1.442) || (abs(Photon_eta[iL]) > 1.556 && abs(Photon_eta[iL]) < 2.5)") \
#          .Filter("(abs(Photon_eta[iS]) < 1.442) || (abs(Photon_eta[iS]) > 1.556 && abs(Photon_eta[iS]) < 2.5)")
df4 = df3.Filter("( (abs(Photon_eta[iL]) < 1.442) || ((abs(Photon_eta[iL]) > 1.566) && (abs(Photon_eta[iL]) < 2.5)) )") \
         .Filter("( (abs(Photon_eta[iS]) < 1.442) || ((abs(Photon_eta[iS]) > 1.566) && (abs(Photon_eta[iS]) < 2.5)) )")

n3 = df4.Count()


# 5. mvaID cuts
#uses terenary operator 

df5 = df4.Filter("(abs(Photon_eta[iL]) < 1.442 ? Photon_mvaID[iL] > -0.02 : Photon_mvaID[iL] > -0.26)").Filter("(abs(Photon_eta[iS]) < 1.442 ? Photon_mvaID[iS] > -0.02 : Photon_mvaID[iS] > -0.26)")
n4 = df5.Count()


# 6. Pixel seed veto  

df6 = df5.Filter("Photon_pixelSeed[iL] == 0 && Photon_pixelSeed[iS] == 0")
n5 = df6.Count()


# Diphoton mass

ROOT.gInterpreter.Declare("""
#include <Math/Vector4D.h>
using V = ROOT::Math::PtEtaPhiMVector;
float dipho_mass(float pt1,float eta1,float phi1,
                 float pt2,float eta2,float phi2){
    V p1(pt1, eta1, phi1, 0.0);
    V p2(pt2, eta2, phi2, 0.0);
    return (p1 + p2).M();
}
""")

df_final = df6.Define("M_lead_sub",
               "dipho_mass(Photon_pt[iL], Photon_eta[iL], Photon_phi[iL], "
               "Photon_pt[iS], Photon_eta[iS], Photon_phi[iS])")

Nf = df_final.Count().GetValue()


# Histogram (DEFINE h)

h = df_final.Histo1D(
    ("M_lead_sub", "Diphoton invariant mass", 120, 0, 200),
    "M_lead_sub"
)


# Draw histogram
h.SetLineColor(ROOT.kBlue)
h.SetLineWidth(2)
c = ROOT.TCanvas("c", "Diphoton mass", 1600, 1200)
ROOT.gStyle.SetOptStat(111111)

h.Draw("HIST")



max_val = h.GetMaximum()
h.SetMaximum(max_val*3)




cuts = ROOT.TPaveText(0.33, 0.55, 0.52, 0.88, "NDC")
cuts.SetFillColor(0)
cuts.SetBorderSize(1)
cuts.SetTextSize(0.025)
cuts.SetTextAlign(12)

cuts.AddText("Photon Selection:")
cuts.AddText("nPhotons>= 2")
cuts.AddText("Lead p_{T} > 30 GeV")
cuts.AddText("Sublead p_{T} > 18 GeV")
cuts.AddText("|#eta| < 1.442 (Barrel)")
cuts.AddText("1.556 < |#eta| < 2.5 (Endcap)")
cuts.AddText("mvaID_{Barrel} > -0.02")
cuts.AddText("mvaID_{Endcap} > -0.26")
cuts.AddText("pixelSeed = False")


cuts.Draw()


# Cut table

cutflow = ROOT.TPaveText(0.15, 0.55, 0.30, 0.88, "NDC")
cutflow.SetFillColor(0)
cutflow.SetBorderSize(1)
cutflow.SetTextSize(0.025)
cutflow.SetTextAlign(12)

cutflow.AddText("Cuts:")

cutflow.AddText(f"Total events: {n0.GetValue()}")
cutflow.AddText(f">=2 photons: {n1.GetValue()}")
cutflow.AddText(f"pT cuts: {n2.GetValue()}")
cutflow.AddText(f"eta cuts: {n3.GetValue()}")
cutflow.AddText(f"mvaID cuts: {n4.GetValue()}")
cutflow.AddText(f"pixelSeed veto: {n5.GetValue()}")

cutflow.AddText(f"Final selected: {Nf}")
cutflow.Draw()


# Save

c.SetGrid()
c.Print("diphoton_with_photonCuts.png")


Error in <TNetXNGFile::Open>: [ERROR] Server responded with an error: [3010] Unable to give access - user access restricted - unauthorized identity used ; Permission denied

Error in <TNetXNGFile::Open>: [ERROR] Server responded with an error: [3010] Unable to give access - user access restricted - unauthorized identity used ; Permission denied

input_line_163:4:7: error: redefinition of 'dipho_mass'
float dipho_mass(float pt1,float eta1,float phi1,
      ^
input_line_91:4:7: note: previous definition is here
float dipho_mass(float pt1,float eta1,float phi1,
      ^
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file diphoton_with_photonCuts.png has been created


In [19]:
n5.GetValue()

8

In [9]:
def photon_preselections_ggH_BBGG_with_cat(
    photons: ak.Array,
    events: ak.Array,
    electron_veto=True,
    revert_electron_veto=False,
    year="2022",
    IsFlag=False):

    print("Number of events before preselection:", len(events))
    print("Year:", year)

    # -------------------------
    # b-tagging working point
    # -------------------------
    if year in ["2022", "2022EE"]:
        wp_medium = 0.2783
    else:
        raise ValueError(f"Unknown year {year}")

    # ------------------------
    # Jet selection
    # ------------------------
    good_jets_mask = (
        (events.Jet.pt > -1.0)
        & (np.abs(events.Jet.eta) < 2.4)
    )
    good_jets = events.Jet[good_jets_mask]

    # b-tag working points
    deepflav_mask = good_jets_mask & (events.Jet.btagDeepFlavB > wp_medium)
    deepflav_bjets = events.Jet[deepflav_mask]

    upar_mask = good_jets_mask & (events.Jet.btagUParTAK4probbb > 0.38)
    upar_bjets = events.Jet[upar_mask]

    # -------------------------------
    # Orthogonal Categories
    # -------------------------------
    cat1 = ak.num(deepflav_bjets) >= 2
    cat2 = (ak.num(upar_bjets) >= 1) & (~cat1)
    cat3 = (ak.num(good_jets) == 1) & (~cat1) & (~cat2)

    # =====================================================
    # ROOT-style photon selection (NO pre-filtering!)
    # =====================================================

    # 1️⃣ Sort photons
    sorted_idx = ak.argsort(photons.pt, ascending=False)
    pho_sorted = photons[sorted_idx]

    # 2️⃣ Pad so iL/iS always exist
    pho_padded = ak.pad_none(pho_sorted, 2)
    iL = pho_padded[:, 0]
    iS = pho_padded[:, 1]

    # 3️⃣ Fill None → safe values
    lead_pt = ak.fill_none(iL.pt, -999)
    sub_pt  = ak.fill_none(iS.pt, -999)

    lead_eta = ak.fill_none(iL.eta, 0)
    sub_eta  = ak.fill_none(iS.eta, 0)

    lead_mva = ak.fill_none(iL.mvaID, -999)
    sub_mva  = ak.fill_none(iS.mvaID, -999)

    lead_pix = ak.fill_none(iL.pixelSeed, 1)
    sub_pix  = ak.fill_none(iS.pixelSeed, 1)

    # 4️⃣ photon multiplicity
    has_two_photons = ak.num(pho_sorted) >= 2

    # 5️⃣ pT cuts
    pt_cut = (lead_pt > 30) & (sub_pt > 18)

    # 6️⃣ eta acceptance
    abs_eta_L = np.abs(lead_eta)
    abs_eta_S = np.abs(sub_eta)

    eta_cut_L = ((abs_eta_L < 1.442) |
                 ((abs_eta_L > 1.556) & (abs_eta_L < 2.5)))
    eta_cut_S = ((abs_eta_S < 1.442) |
                 ((abs_eta_S > 1.556) & (abs_eta_S < 2.5)))

    # 7️⃣ MVAID cuts
    mva_cut_L = ak.where(abs_eta_L < 1.442, lead_mva > -0.02, lead_mva > -0.26)
    mva_cut_S = ak.where(abs_eta_S < 1.442, sub_mva > -0.02, sub_mva > -0.26)

    # 8️⃣ pixel seed
    pixel_cut = (lead_pix == 0) & (sub_pix == 0)

    # 9️⃣ Final ROOT photon mask
    photon_event_mask = (
        has_two_photons &
        pt_cut &
        eta_cut_L & eta_cut_S &
        mva_cut_L & mva_cut_S &
        pixel_cut
    )

    # =====================================================
    # Final category masks including photon selection
    # =====================================================
    mask1 = cat1 & photon_event_mask
    mask2 = cat2 & photon_event_mask
    mask3 = cat3 & photon_event_mask

    print(f"Cat1 events: {ak.sum(mask1)}")
    print(f"Cat2 events: {ak.sum(mask2)}")
    print(f"Cat3 events: {ak.sum(mask3)}")

    # =====================================================
    # Final Output: EXACTLY TWO PHOTONS PER EVENT
    # =====================================================
    two_photons = pho_sorted[:, :2]   # lead/sublead only

    empty = ak.Array([[]] * len(events))

    photons_cat1 = ak.where(mask1, two_photons, empty)
    photons_cat2 = ak.where(mask2, two_photons, empty)
    photons_cat3 = ak.where(mask3, two_photons, empty)

    jets_cat1 = ak.where(mask1, deepflav_bjets, empty)
    jets_cat2 = ak.where(mask2, upar_bjets, empty)
    jets_cat3 = ak.where(mask3, good_jets, empty)

    return {
        "cat1": (photons_cat1, jets_cat1),
        "cat2": (photons_cat2, jets_cat2),
        "cat3": (photons_cat3, jets_cat3),
    }

In [19]:
def photon_preselections_ggH_BBGG(
    photons: ak.Array,
    events: ak.Array,
    electron_veto=True,
    revert_electron_veto=False,
    year="2022",
    IsFlag=False):
    """
    Apply full preselection on leptons, jets, and photons.
    Finally return only photons from events that pass all criteria.
    """

    print("Number of events before preselection:", len(events))
    print("Year:", year)

    #-------------------------
    # b-tagging working point
    #-------------------------
    if year == "2022":
        wp_medium = 0.2783
    elif year == "2022EE":
        wp_medium = 0.2783
    else:
        raise ValueError(f"Unknown year {year}")

    gen = events.GenPart

    bquarks = gen[abs(gen.pdgId) == 5]
    mother_idx = bquarks.genPartIdxMother
    from_a_mask = gen[mother_idx].pdgId == 35
    bquarks_from_a = bquarks[from_a_mask]

    # ------------------------
    # Jet selection
    # ------------------------
    good_jets = (
        (events.Jet.pt > 20.0)
        & (np.abs(events.Jet.eta) < 2.4)
        & (events.Jet.btagDeepFlavB > wp_medium)
    )
    selected_bjets = events.Jet[good_jets]
    at_least_two_bjets = ak.num(selected_bjets) >= 2

    # ------------------------
    # Photon selection
    # ------------------------
    abs_eta = np.abs(photons.eta)

    # Barrel–endcap transition exclusion (1.442 ≤ |η| ≤ 1.566)
    valid_eta = (abs_eta <= 2.5) & ~((abs_eta >= 1.442) & (abs_eta <= 1.566))

    # Barrel vs Endcap ID cuts
    is_barrel = abs_eta < 1.442
    is_endcap = (abs_eta > 1.566) & (abs_eta < 2.5)

    # Apply region-specific MVA thresholds
    barrel_cut = is_barrel & (photons.mvaID > -0.02)
    endcap_cut = is_endcap & (photons.mvaID > -0.26)

    # Base photon selection
    good_photons = (
        (photons.pt > 10)
        & valid_eta
        & (barrel_cut | endcap_cut)
        & (~photons.pixelSeed)
    )
    selected_photons = photons[good_photons]

    # Sort photons by pT in descending order
    selected_photons = selected_photons[ak.argsort(selected_photons.pt, ascending=False)]

    # ------------------------
    # Lead / Sublead cuts
    # ------------------------
    # Define masks for events that have at least 2 photons
    has_two_photons = ak.num(selected_photons) >= 2

    # Safe indexing: fill with -inf if photon is missing
    lead_pt = ak.fill_none(ak.firsts(selected_photons.pt), -999)
    sublead_pt = ak.fill_none(ak.pad_none(selected_photons.pt, 2)[:, 1], -999)

    lead_cut = lead_pt > 30
    sublead_cut = sublead_pt > 18

    # Event must have 2 photons and satisfy both pT cuts
    photon_event_mask = has_two_photons & lead_cut & sublead_cut
    # photon_event_mask = has_two_photons & lead_cut

    # ------------------------
    # Combine jet + photon criteria
    # ------------------------
    event_mask = at_least_two_bjets & photon_event_mask
    # event_mask = photon_event_mask

    # Prepare empty arrays for masked-out events
    empty_photons = ak.Array([[]] * len(events))
    empty_bjets = ak.Array([[]] * len(events))
    empty_bquarks = ak.Array([[]] * len(events))

    # Keep only events passing full selection
    filtered_photons = ak.where(event_mask, selected_photons, empty_photons)
    filtered_jets = ak.where(event_mask, selected_bjets, empty_bjets)
    gen_bquark = ak.where(event_mask, bquarks_from_a, empty_bquarks)

    gen_bquark = gen_bquark[ak.num(gen_bquark.pt)>0]

    print(f"Events passing full preselection: {ak.sum(event_mask)}")

    return filtered_photons, filtered_jets, gen_bquark

In [10]:
import numpy as np
Result = photon_preselections_ggH_BBGG_with_cat(events.Photon, events)

Number of events before preselection: 95231
Year: 2022
Cat1 events: 444
Cat2 events: 2055
Cat3 events: 95


In [20]:
fp, fj, gbq = photon_preselections_ggH_BBGG(events.Photon, events)

Number of events before preselection: 95231
Year: 2022
Events passing full preselection: 294


In [34]:
fp[ak.num(fp.pt)>0].pixelSeed

<Array [[False, False], ..., [False, ...]] type='294 * var * bool[parameter...'>

In [21]:
fp.pt

<Array [[], [], [], [], ..., [], [], [], []] type='95231 * var * float32[pa...'>

In [13]:
photons_cat1 , jets_cat1 = Result["cat1"]

In [44]:
photons_cat1[ak.num(photons_cat1.pt)>0].pixelSeed[150:180]

<Array [[False, False], ..., [False, ...]] type='30 * var * bool[parameters...'>

In [22]:
test1 = ak.from_parquet("/eos/user/b/bbapi/My_Analysis/NTuples_ggH_ptcut_first_with_cat/btagged/ggH_M15-Run3Summer2022NanoAODv12/nominal/*.parquet")

In [37]:
test1.pholead_mvaID

<Array [-0.551, 0.891, -0.499, ..., 0.884, 0.261, 0.75] type='13079 * ?float32'>